# MQTT

MQTT Discovery and updates for homeassistant.

`/home/mosquitto/config/mosquitto.conf`

```yaml
allow_anonymous true
listener 1883 0.0.0.0
```

In [8]:
import asyncio
import aiomqtt

URL = "10.39.10.125"
USER = "leaves"
PWD = "MapleTreeLeaf1492"

async def counter(client: aiomqtt.Client):
    for i in range(10):
        print(f"Publishing {i}")
        await client.publish("counter/count", payload=i)
        await asyncio.sleep(0.1)

async def subscriber(client: aiomqtt.Client):
    await client.subscribe("#")
    async for message in client.messages:
        print(message.payload)

# async with aiomqtt.Client(URL, username=USER, password=PWD) as client:
async with aiomqtt.Client(URL) as client:
    try:
        await counter(client)
        await subscriber(client)
    except asyncio.CancelledError:
        pass


MqttError: timed out

In [7]:
import asyncio
import json
import aiomqtt

msg = {
  "name": "turned on",
  "state_topic": "pump/timestamp_on",
  "device_class": "timestamp",
  "value_template": "{{ as_datetime(value) }}",
  "unique_id": "hp_1231232_ts_on",
  "device": {
    "name": "Heat pump",
    "identifiers": [
      "hp_1231232"
    ]
  }
}

URL = "10.39.10.125"
USER = "leaves"
PWD = "MapleTreeLeaf1492"

async with aiomqtt.Client(URL, username=USER, password=PWD) as client:
    # await client.publish("homeassistant/sensor/hp_1231232_ts_on/config", payload=json.dumps(msg))
    # await asyncio.sleep(SLEEP)
    await client.publish("pump/timestamp_on", payload=1907294116)

In [1]:
import asyncio
import json
import aiomqtt

# https://www.home-assistant.io/integrations/sensor/#device-class

msg1 = {
  "name": "TemperatureB",
  "state_topic": "leaf/state/node-freezer-temperature",
  "device_class": "temperature",  # sets icon
  "unique_id": "node-freezer-temperature",
  "object_id": "node-freezer-temperature",
  "device": {
    "name": "FreezerB",
    "identifiers": [
      "node-freezerB"
    ]
  }
}

msg2 = {
  "name": "HumidityB",
  "state_topic": "leaf/state/node-freezer-humidity",
  "device_class": "humidity",  # sets icon
  "unique_id": "node-freezer-humidity",
  "object_id": "node-freezer-humidity",
  "device": {
    "name": "FreezerB",
    "identifiers": [
      "node-freezerB"
    ]
  }
}

URL = "10.0.0.4"
USER = "leaves"
PWD = "MapleTreeLeaf1492"

SLEEP = 0
DISCOVER = True

async with aiomqtt.Client(URL, username=USER, password=PWD) as client:
    if DISCOVER:
        await client.publish("homeassistant/sensor/node-freezer-temperature/config", payload=json.dumps(msg1))
        await asyncio.sleep(SLEEP)
        await client.publish("homeassistant/sensor/node-freezer-humidity/config", payload=json.dumps(msg2))
        await asyncio.sleep(SLEEP)
    await client.publish("leaf/state/node-freezer-temperature", payload=-77)
    await asyncio.sleep(SLEEP)
    await client.publish("leaf/state/node-freezer-humidity", payload=63)
    

In [1]:
from homeassistant.mqtt import MQTT
from eventbus import bus, Sensor, BinarySensor, Switch, Light
import asyncio, json, aiomqtt, sys

URL = "10.0.0.4"
USER = "leaves"
PWD = "MapleTreeLeaf1492"

bus.unsubscribe_all()
try:
    bus.leaf_id = "leaf_id_2024"
except AssertionError:
    pass

def exception_handler(loop, context):
    print("Exception:", context["message"])

asyncio.get_event_loop().set_exception_handler(exception_handler)




async with aiomqtt.Client(URL, username=USER, password=PWD) as client:
    mqtt = MQTT(client)
    await mqtt.run()
    bs = BinarySensor("door1")

    sw1 = Switch("switch1", manufacturer="Switch & Co", model="Turd")

    light1 = Light("light1", manufacturer="Light & Co", model="Lamp")
    light2 = Light("light2", dimmable=True, manufacturer="Light & Co", model="Lamp")

    climate_sensor = Sensor(
        "climate",
        [
            Sensor.declare_state("temperature", unit_of_measurement="°C"),
            Sensor.declare_state("humidity", unit_of_measurement="%"),
        ],
        manufacturer="MapleTree",
        model="Leaf",
        sw_version="1.0",
    )

    await asyncio.sleep(1)
    await sw1.set('ON')
    # this works!
    # await client.publish("leaf/state/leaf_id_2024-switch1-state", "ON")
    if True:
        for i in range(1000):
            await asyncio.sleep(1)
            await climate_sensor.update("temperature", -18+10*i)
            await climate_sensor.update("humidity", 50-10*i)
            await bs.update(i%2)
            await sw1.set('OFF' if i%2 else 'ON')

DISCOVER BinarySensor(door1) with Entity(leaf_id_2024-door1-state)
DISCOVER Switch(switch1) with Entity(leaf_id_2024-switch1-state), Action(leaf_id_2024-switch1-set)
DISCOVER Light(light1) with Entity(leaf_id_2024-light1-state), Action(leaf_id_2024-light1-set)
DISCOVER Light(light2) with Entity(leaf_id_2024-light2-state), Action(leaf_id_2024-light2-set), Entity(leaf_id_2024-light2-brightness)
DISCOVER Sensor(climate) with Entity(leaf_id_2024-climate-temperature), Entity(leaf_id_2024-climate-humidity)
Action.fire event leaf_id_2024-switch1-set ON
Simulating switch action for leaf_id_2024-switch1-set ON
MQTT PUB leaf/state/leaf_id_2024-switch1-state ON
MQTT PUB leaf/state/leaf_id_2024-climate-temperature -18
MQTT PUB leaf/state/leaf_id_2024-climate-humidity 50
MQTT PUB leaf/state/leaf_id_2024-door1-state OFF
Action.fire event leaf_id_2024-switch1-set ON
Simulating switch action for leaf_id_2024-switch1-set ON
MQTT PUB leaf/state/leaf_id_2024-climate-temperature -8
MQTT PUB leaf/state/lea

In [ ]:
import asyncio
import json
import aiomqtt

# https://www.home-assistant.io/integrations/sensor/#device-class

msg1 = {
  "name": "Temperature",
  "state_topic": "leaf/state/node-freezer-temperature",
  "device_class": "temperature",  # sets icon
  "unique_id": "node-freezer-temperature",
  "object_id": "node-freezer-temperature",
  "device": {
    "name": "Freezer",
    "identifiers": [
      "node-freezer"
    ]
  }
}

msg2 = {
  "name": "Humidity",
  "state_topic": "leaf/state/node-freezer-humidity",
  "device_class": "humidity",  # sets icon
  "unique_id": "node-freezer-humidity",
  "object_id": "node-freezer-humidity",
  "device": {
    "name": "Freezer",
    "identifiers": [
      "node-freezer"
    ]
  }
}

async with aiomqtt.Client("mqtt") as client:
    # discover
    await client.publish("homeassistant/sensor/node-freezer-temperature/config", payload=json.dumps(msg1))
    await client.publish("homeassistant/sensor/node-freezer-humidity/config", payload=json.dumps(msg2))
    # update
    for i in range(10):
        await client.publish("leaf/state/node-freezer-temperature", payload=i)
        await client.publish("leaf/state/node-freezer-humidity", payload=-i)
        await asyncio.sleep(1)
    # remove
    await client.publish("homeassistant/sensor/node-freezer-temperature/config")
    await client.publish("homeassistant/sensor/node-freezer-humidity/config")